**今回は、ヒートアイランド解析の第一歩として、京都市の Landsat データを用いた地表面温度（LST）算出の課題を提供します。**
---

### **目的**
京都市の特定の時期における<span style='color:red'>地表面温度（LST）</span>をLandsat 8またはLandsat 9の熱赤外バンドデータから算出し、その空間分布を可視化する。

### **対象地域・期間**
- <span style='color:red'>地域 (Area of Interest, AOI)</span>: 京都市（事前にGEE上でポリゴンを作成するか、適切な行政区画データを利用してください）。

- <span style='color:red'>衛星データ</span>: Landsat 8 Collection 2 Tier 1 または Landsat 9 Collection 2 Tier 1

- <span style='color:red'>期間</span>: 2023年7月〜8月のうち、雲被覆率が10%未満の画像1枚を選択する。



# **課題：京都市のLandsatデータによる地表面温度（LST）算出**

---
## **応用・発展課題**

**応用課題 1:** 

**NDVIとの相関解析**
ヒートアイランド現象は、植生（NDVI）と密接に関連します。

1. 上記で選択した同じ画像を使って**正規化植生指標（NDVI）**を算出します。

   $$NDVI = \frac{NIR - Red}{NIR + Red}$$

   - Landsat 8/9 のバンドは $SR\_B5$ (NIR) と $SR\_B4$ (Red) を使用します。
   - （ヒント: $L2\_ST$ コレクションは $L2\_SR$ コレクションと同じ画像から生成できます。$L2\_SR$ のバンド $SR\_B4$, $SR\_B5$ を使ってください。または、GEEの組み込み関数 $normalizedDifference(['SR\_B5', 'SR\_B4'])$ を使います。）
   -
2. 算出したLST画像とNDVI画像を合成（$ee.Image.cat()$）し、それぞれの平均値、最大値、最小値を取得します。

3. LSTとNDVIの間の相関係数を計算し、結果をコンソールに出力します。


**応用課題 2:**
   
**異なる年の比較**
京都市のヒートアイランドの変化を追跡します。

1. 2013年（Landsat 8が運用開始した初期）の**同じ時期（7月〜8月）**で、雲被覆率が低い画像を選択します。

2. 2013年と2023年のLSTをそれぞれ算出し、並べて表示します。

3. $2023年LST - 2013年LST$ の差分画像を計算し、地表面温度がこの10年でどれだけ変化したかを可視化します。

---
**これらの課題に取り組むことで、データのフィルタリング、画像演算、結果の可視化というGEEの基本操作を習得できます。**

まずは上記の<span style='color:red'>「LST算出」</span>から始めてみましょう。次のステップとして、コードが完成したら私に教えてください。さらに次の実務に即した課題（例：特定の地目におけるLSTの傾向分析や水域（NDWI）抽出**など）を提供します。

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ee.Initialize(project='earth-change-analysis')

In [ ]:
# 1.AOIの設定（例：簡単な矩形）
# 京都市周辺のざっくりとした範囲135.6, 34.9, 135.9, 35.1
kyoto_aoi = ee.Geometry.Rectangle([135.6, 34.9, 135.9, 35.1])    

# center = ee.Geometry.Point([135.76222, 35.02528])
# kyoto_aoi = center.buffer(20000).bounds()

# 2.Landsat 8 Collection 2 Tier 1 を読み込み
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(kyoto_aoi)\
    .filterDate('2023-07-01', '2023-08-31')\
    .filterMetadata('CLOUD_COVER', 'less_than', 10)

# フィルタリングされたコレクションから1枚の画像を選択
# 雲被覆率が最も低い画像を選択するのも良い方法です
image = l8.sort('CLOUD_COVER').first()

In [ ]:
LST = image.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('LST_Celsius')

In [ ]:
LST_vis = {
    'min' : 20,   # 最小温度（例：20°C）
    'max' : 40,   # 最大温度（例：40°C）
    'palette' : [ # 寒色から暖色へのパレット
        '000080',  # 濃い青
        '0000ff',  # 青
        '00ffff',  # シアン
        '00ff00',  # 緑
        'ffff00',  # 黄色
        'ff0000',  # 赤
        '800000'   # 濃い赤
    ]
}


In [ ]:
map = geemap.Map()
map.set_center(135.76, 35.03, 10)    #  135.76, 35.02, 10

legend_dict = {
    '20°C': '#000080',
    '23°C': '#0000ff',
    '26°C': '#00ffff',
    '29°C': '#00ff00',
    '32°C': '#ffff00',
    '35°C': '#ff0000',
    '40°C': '#800000'
}

map.add_legend(title="LST (°C)", legend_dict=legend_dict)

map.add_layer(LST, LST_vis, 'Kyoto LST (°C)')
map

### **TOAとは「Top of Atmosphere」の略で、Landsat衛星データにおいて大気上端での放射輝度や反射率を表す処理レベルのことです。つまり、衛星センサーが観測した値を大気の影響を含んだまま、物理量（輝度や反射率）に変換したものです。**
---
### **詳細解説**
**<span style='color:green'>TOA反射率 (Top of Atmosphere Reflectance)</span>**
- 太陽光が地表に届き、反射して衛星に戻るまでに大気を通過します。
- このとき、大気による散乱や吸収の影響を含んだ状態での「見かけの反射率」がTOA反射率です。
- Landsat Level-1データは、デジタルナンバー（DN値）を放射輝度やTOA反射率に変換できるようにメタデータに係数が含まれています。
-
**<span style='color:green'>TOA輝度温度 (Top of Atmosphere Brightness Temperature)</span>**
- 熱赤外バンドでは、DN値をTOA輝度温度に変換します。これは大気を通過した後にセンサーが受け取る放射の温度換算値です。

**<span style='color:green'>TOAとSurface Reflectanceの違い</span>**
- **TOA反射率**: 大気の影響を含んだ値。センサーが直接観測した物理量。
- **Surface Reflectance (地表反射率)**: 大気補正を行い、大気の散乱や吸収を取り除いた値。地表の真の反射率に近い。
- 研究や応用では、植生指数（NDVIなど）や土地被覆分類に使う場合は`Surface Reflectance`が望ましいですが、`TOA`は前処理や簡易解析に利用されます。
---
### **まとめ**
- **TOA = 大気上端での反射率・輝度温度**
- **特徴: 大気の影響を含んだままの物理量。
- **用途**: 基本的な解析や、後続の大気補正処理の入力。
- **違い**: Surface ReflectanceはTOAからさらに補正して地表の真の反射率を推定したもの。

Seiichiさんのように都市緑化や災害リスク分析でLandsatを使う場合、TOAは「観測値そのまま」なので、比較や時系列解析には`Surface Reflectance`に変換して使うのが一般的です。もしGoogle Earth Engineで扱うなら、`LANDSAT/LC08/C02/T1_TOA` が`TOA`データ、`LANDSAT/LC08/C02/T1_SR `が`Surface Reflectance`データとして提供されています。
次のステップとして、TOAからSurface Reflectanceへ変換する処理フローを整理してみますか？

---


In [ ]:
# red = 'SR_B4'  nir = 'SR_B5'
'''
def ndvi_image(image):
    nir = image.select('SR_B5')
    red = image.select('SR_B4')
    ndvi = nir.subtract(red).divide(nir.add(red))
    return image.addBands(ndvi)
'''
nirBand = 'SR_B5'
redBand = 'SR_B4'

NDVI = image.normalizedDifference([nirBand, redBand]).rename('NDVI')

ndvi_vis_params = {
    'bands': ['NDVI'],                  # 修正点1: 'NDVI'バンドを選択
    'min': 0.0,                        # 修正点2: 最小値を-1.0から-0.2付近に調整
    'max': 0.5,                         # 修正点3: 最大値を+1.0から0.8付近に調整
    # 水 (-1) から不健康な植生 (0) を経て、健康な植生 (+1) へと変化するパレットを設定
    'palette': ['blue', 'white', 'yellow', 'green', 'darkgreen']
}

map.add_layer(NDVI, ndvi_vis_params, 'NDVI')
map

## **応用課題 1:**

In [ ]:
'''
ee.Image.cat() でバンドを結合
ee.Image.cat() は複数の ee.Image をバンド単位で結合する関数
結果は、2バンドの画像（例：LST_Celsius, NDVI）
'''
combined = LST.addBands(NDVI)

In [ ]:
stats = combined.reduceRegion(
    reducer = ee.Reducer.median()\
        .combine(ee.Reducer.max(), '', True)\
        .combine(ee.Reducer.min(), '', True),
    geometry = kyoto_aoi,
    scale = 30,
    maxPixels  = 1e9
)

# Earth Engine から統計結果を取得
stats_dict = stats.getInfo()

# DataFrame に変換（1行の表として）
df = pd.DataFrame([stats_dict])

# 表示（小数点4桁で整形）
print("統計結果（pandas）:")
print(df.round(4))

In [ ]:
correlation = combined.reduceRegion(
    reducer = ee.Reducer.pearsonsCorrelation(),
    geometry = kyoto_aoi,
    scale = 30,
    maxPixels = 1e9
)

correlation_dict = correlation.getInfo()

# DataFrame に変換（1行の表として）
correlation_df = pd.DataFrame([correlation_dict])

# 表示（小数点4桁で整形）
print("LSTとNDVIの相関係数（pandas）:")
print(correlation_df.round(4))

## **LSTとNDVIの相関分析：結果の解釈**
### **1. 分析の目的**
本分析では、Landsatデータを用いて算出した地表面温度（LST: Land Surface Temperature）と植生指数（NDVI: Normalized Difference Vegetation Index）の間に、どのような関係があるかを統計的に評価することを目的とする。

---
### **2. 相関係数とp値の定義**
- **相関係数（Pearson's r）**
  2つの変数間の線形関係の強さと方向を示す指標。
  値の範囲は -1〜1 で、以下のように解釈される：

|相関係数の範囲	|解釈        |
|---------------|-----------|
|±0.00〜±0.19	|ほぼ無相関   |
|±0.20〜±0.39	|弱い相関     |
|±0.40〜±0.69	|中程度の相関 |
|±0.70〜±1.00	|強い相関     |

p値（有意確率） 　観測された相関が偶然に生じた可能性を示す指標。 　
一般に p < 0.05 の場合、統計的に有意と判断される。 

---
### **3. 結果の概要**

```python
correlation: 0.3921
p-value: 0
```
- 相関係数 `0.3921 `は、NDVIとLSTの間に**弱い正の相関がある**ことを示している。

- p値が `0`（≪ 0.05）であるため、この相関は**統計的に有意**であると判断される。

---
### **4. 解釈と考察**
- NDVIが高い（植生が豊か）な場所では、LSTもやや高くなる傾向が見られた。

- これは、対象地域が農地や湿地など、植生が豊かでも日射を受けやすい環境である可能性を示唆している。

- 一方で、都市部ではNDVIが高いほどLSTが低くなる負の相関が観察されることも多く、地域特性に応じた解釈が必要である。

---
### **5. 結論**
本分析により、NDVIとLSTの間に**弱いが有意な正の相関が存在する**ことが確認された。 今後は、**季節別・土地利用別の相関分析**や、**空間的な相関分布の可視化**を行うことで、より詳細な環境評価が可能となる。



## **応用課題 2:**

### **① データ準備：Landsat 8の画像を取得**

In [ ]:
l8_base_collection= ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(kyoto_aoi)\
    .filterDate('2013-07-01', '2023-08-31')\
    .filterMetadata('CLOUD_COVER', 'less_than', 10)

l8_2013 = l8_base_collection.filterDate('2013-07-01', '2013-08-31')
l8_2023 = l8_base_collection.filterDate('2023-07-01', '2023-08-31')

image_2013 = l8_2013.sort('CLOUD_COVER').first()
image_2023 = l8_2023.sort('CLOUD_COVER').first()

LST_2013 = image_2013.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('2013_LST_Celsius')
LST_2023 = image_2023.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('2023_LST_Celsius')

### **② LST（地表面温度）の計算**

In [ ]:
def calc_LST(image, year):
    return image.select('ST_B10')\
        .multiply(0.00341802).add(149.0).subtract(273.15)\
        .rename(f'{year}_LST_Celsius')

LST_2013 = calc_LST(image_2013, '2013')
LST_2023 = calc_LST(image_2023, '2023')

### **③ NDVI（植生指数）の計算**

In [ ]:
def calc_NDVI(image, year):
    return image.normalizedDifference(['SR_B5', 'SR_B4'])\
        .rename(f'{year}_NDVI')

NDVI_2013 = calc_NDVI(image_2013, '2013')
NDVI_2023 = calc_NDVI(image_2023, '2023')

### **④ 温度差・NDVI差のマップを作成**

In [ ]:
LST_diff = LST_2023.subtract(LST_2013).rename('LST_Diff')
NDVI_diff = NDVI_2023.subtract(NDVI_2013).rename('NDVI_Diff')

### **⑤ 平均値を取得（reduceRegion）**

In [ ]:
# LSTの平均
mean_LST_2013 = LST_2013.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=kyoto_aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

mean_LST_2023 = LST_2023.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=kyoto_aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

# NDVIの平均
mean_NDVI_2013 = NDVI_2013.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=kyoto_aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

mean_NDVI_2023 = NDVI_2023.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=kyoto_aoi,
    scale=30,
    maxPixels=1e9
).getInfo()


### **⑥ 結果を表示（pandasで整形）**

In [ ]:
df = pd.DataFrame([{
    'LST_2013 (°C)': mean_LST_2013['2013_LST_Celsius'],
    'LST_2023 (°C)': mean_LST_2023['2023_LST_Celsius'],
    'NDVI_2013': mean_NDVI_2013['2013_NDVI'],
    'NDVI_2023': mean_NDVI_2023['2023_NDVI'],
    'LST_Change (°C)': mean_LST_2023['2023_LST_Celsius'] - mean_LST_2013['2013_LST_Celsius'],
    'NDVI_Change': mean_NDVI_2023['2023_NDVI'] - mean_NDVI_2013['2013_NDVI']
}])

print("LSTとNDVIの変化（2013→2023）:")
print(df.round(4))


### **⑦ オプション：差分マップをGEEで可視化**

In [ ]:
map.set_center(135.76, 35.03, 10)

# 表示パラメータ
lst_vis = {'min': -5, 'max': 5, 'palette': ['blue', 'white', 'red']}
ndvi_vis = {'min': -0.3, 'max': 0.3, 'palette': ['purple', 'white', 'green']}

map.add_layer(LST_diff, lst_vis, 'LST Change (°C)')
map.add_layer(NDVI_diff, ndvi_vis, 'NDVI Change')

# 凡例を追加
# 凡例を追加（colorsを明示的に指定！）
map.add_colorbar_branca(
    colors=lst_vis['palette'],
    vmin=lst_vis['min'],
    vmax=lst_vis['max'],
    label='LST Change (°C)'
)

map.add_colorbar_branca(
    colors=ndvi_vis['palette'],
    vmin=ndvi_vis['min'],
    vmax=ndvi_vis['max'],
    label='NDVI Change'
)

map


## **課題のフィードバックと分析** 

1. **LSTとNDVIの相関解析のフィードバック**
  
|指標                |結果                 |分析                                                        |
|-------------------|---------------------|-----------------------------------------------------------|
|**LSTの中央値**          |$31.12^{\circ}C$    |猛暑日の京都市の夏の温度として妥当な値です。                       |
|**NDVIの中央値**         |$0.2246$            |都市域（低植生）と山間部（高植生）が混在するAOIの中央値として適切です。|
|**LST最大値**           |$51.14^{\circ}C$     |**非常に高い値**です。これは、**アスファルト、屋根などの熱容量の高い地表面**や、水蒸気の影響を受けやすい場所などが考えられます。ヒートアイランドのコアな部分を示唆します。|
|**相関係数**             |$0.3186$            |**弱い正の相関**を示しています。                                     |


**【分析と考察】**

- **LSTとNDVIは一般的に強い負の相関（NDVIが高い＝植生が多い＝LSTが低い）を示す**ことが多いですが、結果は$0.3186$という**弱い正の相関**となりました。

- これは、京都市のAOI（おそらく広範囲で山間部を含む）における**特定の日付の単一画像**が持つ大気の不均一性や、<span style='color:black'><strong>雲や影のピクセル（NDVIが低い）がLSTの異常値（特に低温側）</strong></span>に影響を与えている可能性があります。

- 実務では、この「**想定と異なる結果**」が出た際に、異常なピクセルを除外するクラウドマスキング（$Fmask$や$QA$バンドの利用）や、**複数画像の平均値**を使うなどのロバストな前処理が必要になります。

---

2. **LSTとNDVIの10年間の変化のフィードバック**

|指標      |2013年平均         |2023年平均        |変化量（2023 - 2013） |
|---------|------------------|-----------------|---------------------|
|LST平均   |$39.80^{\circ}C$  |$30.39^{\circ}C$ |$-9.41^{\circ}C$     |
|NDVI平均  |$0.2444$          |$0.2250$         |$-0.0194$            |


**【分析と考察】**

- LSTの変化が**$-9.41^{\circ}C$と、非常に大きな減少を示しています。これは京都の気温が10年で大幅に上昇したという事実とは矛盾**します。

- 原因は、**比較対象の2枚の画像の日付と時刻、特に大気の状態が大きく異なっている**ためです。

    - 2013年の画像は、**非常に暑い日の正午付近**で、**乾燥した**大気下で撮影された可能性があります（LSTが高く算出されやすい）。

    - 2023年の画像は、**やや曇りの日の午前中**や**湿度の高い日**に撮影された可能性があります（LSTが低く算出されやすい）。
 
- 実務的な課題として、この問題を解決するためには、**比較する画像間のバイアス**を減らす工夫が必要です。

---

## **Git commit message**

```bash
Add GEE notebook: 京都市のLandsatデータによる地表面温度（LST）算出 (2025-12-01)

- ファイル名: 0011_kyoto_LST_NDVI_20251201.ipynb
- 概要: LSTとNDVIの相関分析
- 使用データ: LANDSAT/LC08/C02/T1_L2
- 手法: LST、NDVIの算出、それぞれの平均値、最低値、最高値を算出、相関係数とP値の算出
- 備考: 図表はMarkdownに分離、出力セルはnbstripoutで除去
```